# **PREPROCESSING**

In [ ]:
# 1) Upload file when prompted
from google.colab import files
import pandas as pd, numpy as np
from pathlib import Path

print("Choose your .xlsx or .csv file…")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")

fname = list(uploaded.keys())[0]
p = Path(f"/content/{fname}")

Choose your .xlsx or .csv file…


RuntimeError: No file uploaded.

In [ ]:
# 2) Loading the uploaded file (Excel or CSV) into a pandas DataFrame and print its size
if p.suffix.lower() == ".xlsx":
    df = pd.read_excel(p)
elif p.suffix.lower() == ".csv":
    df = pd.read_csv(p)
else:
    raise ValueError("Please upload a .xlsx or .csv file.")

print("Loaded:", p.name, df.shape)

Loaded: luxury_cosmetics_popups.xlsx (2133, 15)


In [ ]:
# 3) Basic cleaning & types
cols_wanted = [
    "event_id","brand","region","city","location_type","event_type",
    "start_date","end_date","lease_length_days","sku","product_name",
    "price_usd","avg_daily_footfall","units_sold","sell_through_pct"
]
df = df[[c for c in cols_wanted if c in df.columns]].copy()

# Parse dates if present
for c in ["start_date","end_date"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

# Recompute lease length if dates available
if all(c in df.columns for c in ["start_date","end_date","lease_length_days"]):
    mask = df["start_date"].notna() & df["end_date"].notna()
    lease_from_dates = (df.loc[mask, "end_date"] - df.loc[mask, "start_date"]).dt.days
    df.loc[mask, "lease_length_days"] = lease_from_dates.clip(lower=0)

# Coerce numerics
for c in ["lease_length_days","price_usd","avg_daily_footfall","units_sold","sell_through_pct"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Bounds & duplicates
if "sell_through_pct" in df.columns:
    df["sell_through_pct"] = df["sell_through_pct"].clip(0, 100)
for c in ["price_usd","avg_daily_footfall","units_sold","lease_length_days"]:
    if c in df.columns:
        df[c] = df[c].clip(lower=0)
if "event_id" in df.columns:
    df = df.drop_duplicates(subset=["event_id"])

In [ ]:
# 4) Derived KPIs
df["total_visitors"] = (df.get("avg_daily_footfall", 0) * df.get("lease_length_days", 0)).fillna(0)
df["revenue_est"] = (df.get("units_sold", 0) * df.get("price_usd", 0)).fillna(0)
with np.errstate(divide="ignore", invalid="ignore"):
    df["conversion_est"] = (
      pd.Series(
          np.where(
              df["total_visitors"] > 0,
              df.get("units_sold", 0) / df["total_visitors"],
              np.nan
          ),
          index=df.index
      ).clip(lower=0)
  )


# Sorting by brand and time to ensure correct order for rate-of-change calculations, then reset index
if "start_date" in df.columns:
    df = df.sort_values(["brand","start_date"], kind="mergesort")
else:
    df = df.sort_values(["brand"], kind="mergesort")
df.reset_index(drop=True, inplace=True)

In [ ]:
# 5) Choose the KPI that drives pitch — change to "sell_through_pct"
PRIMARY_VALUE = "units_sold"

df["value_raw"] = df[PRIMARY_VALUE].astype(float)
df["value_roc"] = df.groupby("brand", group_keys=False)["value_raw"].diff()
df["value_roc_abs"] = df["value_roc"].abs().fillna(0)

def minmax(s):
    s = s.astype(float)
    lo, hi = s.min(), s.max()
    if pd.isna(lo) or pd.isna(hi) or hi == lo:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - lo) / (hi - lo)

def zscore(s):
    s = s.astype(float)
    mu, sd = s.mean(), s.std(ddof=0)
    if pd.isna(mu) or pd.isna(sd) or sd == 0:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - mu) / sd

df["value_roc_norm"] = minmax(df["value_roc_abs"])
df["value_norm"] = minmax(df["value_raw"])

In [ ]:
# 6) Audio mappings
df["pitch_hz"] = 220 + df["value_norm"] * (880 - 220)
df["volume"] = df["value_roc_norm"].fillna(0.2)

instrument_bank = ["sine","triangle","square","sawtooth"]
def pick_instrument(name):
    if pd.isna(name): return "sine"
    return instrument_bank[hash(str(name)) % len(instrument_bank)]
df["instrument"] = df.get("brand", pd.Series(["sine"]*len(df))).apply(pick_instrument)

In [ ]:
# 7) Export
OUT_DIR = Path("/content/sonidash_out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Full analysis file: keep EVERYTHING
full_out = OUT_DIR / "luxury_cosmetics_cleaned.csv"
df.to_csv(full_out, index=False)

# Lightweight KPI file: only what the web app needs
sonidash_cols = [
    "brand",            # label / instrument mapping
    "region",           # optional filter/group
    "event_type",       # optional filter
    "start_date",       # optional ordering in UI
    "sell_through_pct", # KPI → pitch (human-readable)
    "units_sold",       # KPI → volume (human-readable)
    "pitch_hz",         # audio param for Tone.js
    "volume",           # audio param for Tone.js
    "instrument"        # audio timbre per brand
]
sonidash_cols = [c for c in sonidash_cols if c in df.columns]
df_soni = df.loc[:, sonidash_cols]  # <- create a filtered copy; do NOT modify df

sonidash_out = OUT_DIR / "SoniDash_KPI.csv"
df_soni.to_csv(sonidash_out, index=False)

print("Exports ready")
print("Full cleaned CSV:", full_out, "(shape:", df.shape, ")")
print("SoniDash CSV:", sonidash_out, "(shape:", df_soni.shape, ")")

# Quick sanity peek
df_soni.head(10)

Exports ready
Full cleaned CSV: /content/sonidash_out/luxury_cosmetics_cleaned.csv (shape: (2133, 26) )
SoniDash CSV: /content/sonidash_out/SoniDash_KPI.csv (shape: (2133, 9) )


,brand,region,event_type,start_date,sell_through_pct,units_sold,pitch_hz,volume,instrument
0,Armani Beauty,Europe,Flash Event,2024-02-25,47.80,805,316.880734,0.000000,triangle
1,Armani Beauty,Asia-Pacific,Mall Kiosk,2024-03-01,60.37,2718,580.137615,0.416413,triangle
2,Armani Beauty,Asia-Pacific,Mall Kiosk,2024-03-04,67.24,1398,398.486239,0.287331,triangle
3,Armani Beauty,Middle East,Mobile Pop-Up Truck,2024-03-12,74.63,806,317.018349,0.128864,triangle
4,Armani Beauty,Asia-Pacific,Mobile Pop-Up Truck,2024-03-17,60.76,675,298.990826,0.028515,triangle
5,Armani Beauty,North America,Mobile Pop-Up Truck,2024-03-17,83.98,713,304.220183,0.008272,triangle
6,Armani Beauty,Asia-Pacific,Mall Kiosk,2024-03-27,58.20,2086,493.165138,0.298868,triangle
7,Armani Beauty,Europe,Flash Event,2024-03-31,64.91,1121,360.366972,0.210057,triangle
8,Armani Beauty,Latin America,Standalone Pop-Up,2024-04-06,87.21,4042,762.339450,0.635829,triangle
9,Armani Beauty,Asia-Pacific,Shop-in-Shop,2024-04-15,74.86,1382,396.284404,0.579016,triangle
